# NREL Data Handling
A Python class for handling data from NREL, the National Renewable Energy Laboratory. 

## Features
- Downloads data for a given year or multiple years.
- Get data from file and return it as a pandas DataFrame.
- Get features and target from data.
- Split data into training and testing sets.

## Usage
```python
nrel = NREL()
nrel.download_data_by_year(2022)
nrel.download_data_for_multiple_years([2019, 2020, 2021])

data = nrel.get_data('data/NREL/atb_electricity_2022.csv')
features = nrel.get_features(data)
target = nrel.get_target(data)

train_data, test_data = nrel.split_data(0.2)


In [ ]:
import os
import pandas as pd
from typing import Tuple, List

class NREL:
    """
    Class to handle NREL data.
    """
    def __init__(self):
        self.data = None
        self.years_data = {}
        self.years = []
    
   
    def download_data_by_year(self, year: int) -> None:
        """
        Downloads data for a given year.

        The function uses a dictionary to determine the URL to download data from, based on the year parameter. The data is then downloaded as a CSV and saved to the current working directory.

        Parameters
        ----------
        year : int
            The year for which data should be downloaded. Must be between 2019 and 2022.

        Returns
        -------
        None

        Raises
        ------
        ValueError
            If the year parameter is outside the range of 2019 to 2022.
        """
        # Dictionary mapping year to URL
        year_url = {
            2019: "https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/csv/2019/ATBe.csv",
            2020: "https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/csv/2020/ATBe.csv",
            2021: "https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/csv/2021/ATBe.csv",
            2022: "https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/csv/2022/ATBe.csv"
        }

        # Check if year is in range
        if year not in year_url:
            raise ValueError(f"Year must be between 2019 and 2022. Got {year}")

        # Get URL for year
        url = year_url[year]

        # Create directory if it does not exist
        if not os.path.exists("data/NREL"):
            os.makedirs("data/NREL")
        
        # Download data
        df = pd.read_csv(url)
        df.to_csv(f"data/NREL/atb_electricity_{year}.csv", index=False)
        
        # Store data in NREL object
        self.years.append((year, df))

    def download_data_for_multiple_years(self, years: List[int]) -> pd.DataFrame:
        """
        Downloads data for multiple years and returns a combined DataFrame.

        Parameters
        ----------
        years : list of int
            The years for which data should be downloaded. Each year must be between 2019 and 2022.

        Returns
        -------
        df : pd.DataFrame
            The combined data for all years.

        Raises
        ------
        ValueError
            If any of the years in the years parameter are outside the range of 2019 to 2022.
        """
        for year in years:
            self.download_data_by_year(year)
        
        # Combine data from all years
        df = pd.concat([df for _, df in self.years], axis=0, ignore_index=True)
        
        return df

    def get_data(self, filepath: str) -> pd.DataFrame:
        """
        Get the data from a file and return it as a pandas DataFrame.
        
        :param filepath: The file path of the data.
        :return: A pandas DataFrame containing the data.
        """
        data = pd.read_csv(filepath)
        self.data = data
        return data

    def get_features(self, data: pd.DataFrame = None) -> pd.DataFrame:
        """
        Get the features from the data and return them as a pandas DataFrame.
        
        :param data: A pandas DataFrame containing the data. If not provided, the class data attribute will be used.
        :return: A pandas DataFrame containing the features.
        """
        if data is None:
            data = self.data
        features = data.drop(columns=['target'])
        return features

    def get_target(self, data: pd.DataFrame = None) -> pd.Series:
        """
        Get the target from the data and return it as a pandas Series.
        
        :param data: A pandas DataFrame containing the data. If not provided, the class data attribute will be used.
        :return: A pandas Series containing the target.
        """
        if data is None:
            data = self.data
        target = data['target']
        return target

    def split_data(self, test_size: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Split the data into training and testing sets and return them as a tuple of pandas DataFrames.

        :param test_size: The proportion of the data to use for testing.
        :return: A tuple of pandas DataFrames (training data, testing data).
        """
        #self.training_data, self.testing_data = train_test_split(self.data, test_size=test_size)
        #return self.training_data, self.testing_data

        return

In [ ]:
nrel = NREL()

years_to_download = [2019, 2020, 2021]
df = nrel.download_data_for_multiple_years(years_to_download)


df.sample(frac=1).reset_index(drop=True).head()

<ipython-input-15-82e887b3d4a8>:81: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  self.download_data_by_year(year)


In [ ]:
# Get unique values of column 'atb_year'
print('unique years are: ',df['atb_year'].unique(), '\n\n\n\n')

# Preview the first 5 rows of the shuffled dataframe
df.sample(frac=1).reset_index(drop=True).head()


unique years are:  [2019 2020 2021] 






,Unnamed: 0,revision,atb_year,core_metric_key,core_metric_parameter,core_metric_case,crpyears,technology,techdetail,scenario,core_metric_variable,units,value,update_date,technology_alias,display_name,default
0,168187,13.0,2020,RC3LaLTRG100,Fixed O&M,Market,20,LandbasedWind,LTRG10,Conservative,2037,$/KW-yr,42.453825,NaN,NaN,NaN,NaN
1,170049,NaN,2021,RM3PuClass 4,Variable O&M,R&D,20,Pumped Storage Hydropower,Class 4,Moderate,2020,$/MWh,0.506300,NaN,Pumped Storage Hydropower,Pumped Storage Hydropower Class 4,0.0
2,190106,13.0,2020,RC3OfOTRG88,LCOE,Market,20,OffShoreWind,OTRG8,Conservative,2044,$/MWh,74.476851,NaN,NaN,NaN,NaN
3,158143,NaN,2021,RM3NaCCCCSHF,CAPEX,R&D,55,NaturalGas_FE,CCCCSHighCF,Moderate,2050,$/kW,1699.512293,NaN,Natural Gas,Natural Gas FE CCCCS High CF,1.0
4,121488,NaN,2021,RC3UtClass66,LCOE,R&D,20,Utility-Scale PV-Plus-Battery,Class6,Conservative,2035,$/MWh,49.370815,NaN,Utility-Scale PV-Plus-Battery,Utility-Scale PV+Battery Class 6,0.0


TypeError: ignored